In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
((sqrt(2)-I)^3+(sqrt(2)-I)).expand()

-6*I

In [3]:
(sqrt(2)-I).minpoly('xx')

xx^4 - 2*xx^2 + 9

In [4]:
var('xx')
Q.<rad2_minus_i> = NumberField(xx^4 - 2*xx^2 + 9)

In [5]:
i = ((rad2_minus_i)^3+rad2_minus_i)/(-6)

In [9]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(Q)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE1 = general_cubic.subs({b:-(c*c+e*f)/c})

In [10]:
e_sost = {e: (-4*i)*c + d + (-2*i + 1)*f}
d_sost = {d: ((-64*i - 48)*c^2 + (64*i + 128)*c*f + (64*i + 48)*f^2)/((32*i - 16)*c + (-32*i + 16)*f)}

In [11]:
SE1_1 = SE1.subs(e_sost).subs(d_sost)

In [21]:
%time adm_SE1_1 = SE1_1.find_admissible_projectivities()
len(adm_SE1_1)

CPU times: user 6.02 s, sys: 245 ms, total: 6.26 s
Wall time: 35.1 s


576

In [22]:
%time simm_SE1_1 = SE1_1.find_simmetries(adm_SE1_1)
len(simm_SE1_1)

CPU times: user 6.81 s, sys: 245 ms, total: 7.06 s
Wall time: 19.5 s


4

In [23]:
%time conds = SE1_1.find_conditions_for_subfamilies(adm_SE1_1, simm_SE1_1)

CPU times: user 8min 25s, sys: 348 ms, total: 8min 26s
Wall time: 8min 26s


In [24]:
conds

[[9*c + (-rad2_minus_i^3 + 2*rad2_minus_i)*f],
 [18*c + (-rad2_minus_i^3 + 3*rad2_minus_i^2 - 7*rad2_minus_i + 3)*f],
 [6*c + (rad2_minus_i^3 + 3*rad2_minus_i^2 + rad2_minus_i - 3)*f],
 [6*c + (-rad2_minus_i^3 + rad2_minus_i^2 + rad2_minus_i - 3)*f],
 [102*c + (-7*rad2_minus_i^3 - 15*rad2_minus_i^2 + 11*rad2_minus_i + 9)*f],
 [102*c + (-rad2_minus_i^3 + 15*rad2_minus_i^2 - 19*rad2_minus_i - 21)*f],
 [6*c + (rad2_minus_i^3 - 3*rad2_minus_i^2 + rad2_minus_i + 3)*f],
 [18*c + (-rad2_minus_i^3 - 3*rad2_minus_i^2 - 7*rad2_minus_i - 3)*f],
 [3*c + (-rad2_minus_i)*f],
 [102*c + (-rad2_minus_i^3 - 15*rad2_minus_i^2 - 19*rad2_minus_i + 21)*f],
 [102*c + (-7*rad2_minus_i^3 + 15*rad2_minus_i^2 + 11*rad2_minus_i - 9)*f],
 [6*c + (-rad2_minus_i^3 - rad2_minus_i^2 + rad2_minus_i + 3)*f]]

In [12]:
sosts = [solve_linear_system(cond, [c], [f]) for cond in conds]
cubics = [SE1_1.subs({c:sost[0], f:sost[1]}) for sost in sosts]
#%time [len(cubic.find_simmetries()) for cubic in cubics]

In [31]:
SE1_2 = SE1_1.subs({c: rad2_minus_i*f/3})

In [26]:
%time adm_SE1_2 = SE1_2.find_admissible_projectivities()
len(adm_SE1_2)

CPU times: user 2.54 s, sys: 180 ms, total: 2.72 s
Wall time: 15.8 s


576

In [27]:
%time simm_SE1_2 = SE1_2.find_simmetries(adm_SE1_2)
len(simm_SE1_2)

CPU times: user 2.65 s, sys: 176 ms, total: 2.83 s
Wall time: 3.16 s


8

In [28]:
lines_perms = []
for simm in simm_SE1_2:
    lines_perms.append(Permutation(SE1_2.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(8, 'C8')

In [29]:
[get_permuted_extended_L_set(perm) for perm in lines_perms_group.gens_small()], lines_perms_group.gens_small()

([('G4', 'E1', 'G2', 'E6', 'F15', 'G3')],
 [(1,10)(2,8,22,11,19,13,3,16)(4,27,23,24,7,18,21,20)(5,9,6,17,25,14,26,12)])